In [60]:
#data sources, etc
import pandas as pd
import numpy as np
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import time
from sklearn.metrics import mean_squared_error
from math import sqrt
import torch
import torch.nn as nn
from torch.autograd import Variable

In [4]:
cData=pd.read_csv('127compiledDailyData127.csv') #.drop(columns='Unnamed: 0')
cData

,date,open,high,low,close,volume,ticker
0,2020-06-08,9.7000,9.830,9.4900,9.53,6083313.0,SRTY
1,2020-06-05,9.9600,10.270,9.6500,10.13,11205729.0,SRTY
2,2020-06-04,11.6399,11.780,11.2150,11.46,8088398.0,SRTY
3,2020-06-03,11.7200,11.850,11.0900,11.41,9058773.0,SRTY
4,2020-06-02,12.3800,12.790,12.1700,12.30,6600797.0,SRTY
...,...,...,...,...,...,...,...
12619,2020-01-22,56.8800,57.030,56.6500,56.85,4256610.0,KRE
12620,2020-01-21,57.1400,57.405,56.7901,56.80,5027864.0,KRE
12621,2020-01-17,57.9100,57.950,57.4400,57.54,8222643.0,KRE
12622,2020-01-16,56.9000,57.480,56.8900,57.41,7366494.0,KRE


In [16]:
57.480/56.9000

1.0101933216168717

In [5]:
cData['ticker'].unique()

array(['SRTY', 'CAL', 'TEN', 'OSUR', 'APY', 'SABR', 'OVV', 'CIM', 'RUTH',
       'LILAK', 'UAA', 'MAC', 'TGI', 'SVC', 'MT', 'EB', 'AA', 'GPS',
       'XHR', 'PCG', 'AMKR', 'AZUL', 'ROIC', 'HBI', 'RLJ', 'PK', 'BPYU',
       'DENN', 'JBLU', 'BLMN', 'UGAZ', 'HST', 'KEY', 'FAZ', 'DLPH', 'SRG',
       'CXP', 'RRR', 'YEXT', 'PEB', 'AKR', 'PVAC', 'HOMB', 'ELY', 'DXD',
       'CUK', 'EPRT', 'CNP', 'PPC', 'XRX', 'AAL', 'GAN', 'NMIH', 'PLAY',
       'DFEN', 'SEAS', 'HA', 'OUT', 'VVV', 'TZA', 'JETS', 'OXY', 'ERX',
       'CCL', 'MGM', 'SDOW', 'DBX', 'WRI', 'NCLH', 'CCC', 'JWN', 'SAVE',
       'KTB', 'FSCT', 'USFD', 'AU', 'ILF', 'HGV', 'CHK', 'TRIP', 'PD',
       'CIT', 'HSBC', 'KSS', 'REM', 'YELP', 'HTA', 'CAKE', 'EWG', 'KDP',
       'PEAK', 'GM', 'SCHC', 'CAR', 'APLS', 'WFC', 'NAIL', 'WORK', 'KHC',
       'DUST', 'SPR', 'VTIQ', 'DAL', 'ZTO', 'RWM', 'TMF', 'CUZ', 'COLD',
       'TCBI', 'NKLA', 'KBE', 'WYND', 'DT', 'LUV', 'ADPT', 'URTY', 'ARES',
       'FR', 'WERN', 'UAL', 'ALK', 'SCHW', 'EFV', 'R

In [48]:
def ratioForwardMA (df,ma=5,maCol='open', forward=1):
    compiledDF=pd.DataFrame()
    for ticker in df['ticker'].unique():
        dfSub=df[df['ticker']==ticker]
        dfRatio=pd.DataFrame()
        
        dfRatio['open']=dfSub['open']/dfSub['open'].iloc[-1]
        dfRatio['high']=dfSub['high']/dfSub['open'].iloc[-1]
        dfRatio['low']=dfSub['low']/dfSub['open'].iloc[-1]
        dfRatio['close']=dfSub['close']/dfSub['open'].iloc[-1]
       
    
        dfRatio['upperShadow']=(dfRatio['high']-dfRatio[['open','close']].max(axis=1))
        dfRatio['lowerShadow']=(dfRatio['low']-dfRatio[['open','close']].min(axis=1))
        dfRatio['body']=dfRatio['close']-dfRatio['open']

        dfRatio['openF'+str(forward)]=dfRatio['open'].shift(forward)
        dfRatio['highF'+str(forward)]=dfRatio['high'].shift(forward)
        dfRatio['lowF'+str(forward)]=dfRatio['low'].shift(forward)
        dfRatio['closeF'+str(forward)]=dfRatio['close'].shift(forward)
        
        dfRatio['MA'+str(ma)]=dfRatio[maCol].rolling(window=ma).mean().shift(-ma+1)
        dfRatio.insert(0,'date',dfSub['date'])
        dfRatio.insert(1,'ticker', dfSub['ticker'])
        
        dfRatio=dfRatio.iloc[::-1]
        compiledDF=compiledDF.append(dfRatio)
        
    compiledDF=compiledDF.dropna()
    return compiledDF
        

In [49]:
rfmaDF=ratioForwardMA(cData)
rfmaDF

,date,ticker,open,high,low,close,upperShadow,lowerShadow,body,openF1,highF1,lowF1,closeF1,MA5
95,2020-01-22,SRTY,0.964105,0.980370,0.957375,0.975883,0.004487,-0.006730,0.011778,0.983735,1.010656,0.968873,0.975322,0.962759
94,2020-01-23,SRTY,0.983735,1.010656,0.968873,0.975322,0.026921,-0.006450,-0.008413,0.966910,1.032529,0.966910,1.014582,0.959506
93,2020-01-24,SRTY,0.966910,1.032529,0.966910,1.014582,0.017947,0.000000,0.047672,1.060437,1.067190,1.030286,1.047672,0.961189
92,2020-01-27,SRTY,1.060437,1.067190,1.030286,1.047672,0.006753,-0.017386,-0.012765,1.027482,1.033090,1.011217,1.021312,0.987522
91,2020-01-28,SRTY,1.027482,1.033090,1.011217,1.021312,0.005609,-0.010095,-0.006169,1.016826,1.039260,1.009534,1.039260,1.000534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12529,2020-06-01,KRE,0.675405,0.689127,0.667840,0.678220,0.010908,-0.007565,0.002815,0.689655,0.694757,0.669775,0.676108,0.684483
12528,2020-06-02,KRE,0.689655,0.694757,0.669775,0.676108,0.005102,-0.006334,-0.013547,0.695461,0.723082,0.694053,0.715165,0.693913
12527,2020-06-03,KRE,0.695461,0.723082,0.694053,0.715165,0.007917,-0.001407,0.019704,0.711999,0.741731,0.703730,0.740852,0.693103
12526,2020-06-04,KRE,0.711999,0.741731,0.703730,0.740852,0.000880,-0.008269,0.028853,0.807002,0.811400,0.777973,0.783779,0.690042


In [50]:
rfmaDF.to_csv('127CalculatedDataFrame.csv')

In [51]:
import plotly.graph_objects as go

ticker='SRTY'
dfChart = rfmaDF[rfmaDF['ticker']==ticker]


fig = go.Figure(data=[go.Candlestick(x=dfChart['date'],
                open=dfChart['open'], high=dfChart['high'],
                low=dfChart['low'], close=dfChart['close'])
                     ])

#add xaxis_type='category' and fig.update_xaxes(autorange="reversed") to remove the datetime gaps
fig.update_layout(xaxis_type='category', xaxis_rangeslider_visible=False, title_text=ticker)
#fig.update_xaxes(autorange="reversed")
fig.show()

dfChart=cData[cData['ticker']==ticker]
fig = go.Figure(data=[go.Candlestick(x=dfChart['date'],
                open=dfChart['open'], high=dfChart['high'],
                low=dfChart['low'], close=dfChart['close'])
                     ])

#add xaxis_type='category' and fig.update_xaxes(autorange="reversed") to remove the datetime gaps
fig.update_layout(xaxis_type='category', xaxis_rangeslider_visible=False, title_text=ticker)
fig.update_xaxes(autorange="reversed")
fig.show()

In [56]:
ticker='SRTY'
stock=rfmaDF[rfmaDF['ticker']==ticker][['date', 'open','high','close','low','openF1']]
stock

,date,open,high,close,low,openF1
95,2020-01-22,0.964105,0.980370,0.975883,0.957375,0.983735
94,2020-01-23,0.983735,1.010656,0.975322,0.968873,0.966910
93,2020-01-24,0.966910,1.032529,1.014582,0.966910,1.060437
92,2020-01-27,1.060437,1.067190,1.047672,1.030286,1.027482
91,2020-01-28,1.027482,1.033090,1.021312,1.011217,1.016826
...,...,...,...,...,...,...
5,2020-06-01,0.722378,0.732473,0.707796,0.684801,0.694335
4,2020-06-02,0.694335,0.717330,0.689849,0.682557,0.657319
3,2020-06-03,0.657319,0.664610,0.639933,0.621985,0.652827
2,2020-06-04,0.652827,0.660684,0.642737,0.628996,0.558609


In [63]:
def load_data(stock, look_back):
    data_raw = stock.values # convert to numpy array
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - look_back): 
        data.append(data_raw[index: index + look_back])
    
    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]

look_back = 20 # choose sequence length
x_train, y_train, x_test, y_test = load_data(stock, look_back)
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ',y_train.shape)
print('x_test.shape = ',x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train.shape =  (60, 19, 6)
y_train.shape =  (60, 6)
x_test.shape =  (15, 19, 6)
y_test.shape =  (15, 6)


In [ ]:
# make training and test sets in torch
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)

#train_X = train_X.view([-1, x_train.shape[0], 1])
#test_X = test_X.view([-1, x_test.shape[0], 1])
#train_Y = train_Y.view([y_train.shape[0], 1])

y_train.size(),x_train.size()

In [ ]:
n_steps = look_back-1
batch_size = 1606
#n_iters = 3000
num_epochs = 100 #n_iters / (len(train_X) / batch_size)
#num_epochs = int(num_epochs)

train = torch.utils.data.TensorDataset(x_train,y_train)
test = torch.utils.data.TensorDataset(x_test,y_test)

train_loader = torch.utils.data.DataLoader(dataset=train, 
                                           batch_size=batch_size, 
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [ ]:
# Build model
#####################
input_dim = 1
hidden_dim = 32
num_layers = 2 
output_dim = 1


# Here we define our model as a class
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.num_layers = num_layers

        # Building your LSTM
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # One time step
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        # out.size() --> 100, 28, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out[:, -1, :]) 
        # out.size() --> 100, 10
        return out
    
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)

loss_fn = torch.nn.MSELoss(size_average=True)

optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

In [ ]:
# Train model
#####################

hist = np.zeros(num_epochs)

# Number of steps to unroll
seq_dim =look_back-1  

for t in range(num_epochs):
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    #model.hidden = model.init_hidden()
    
    # Forward pass
    y_train_pred = model(x_train)

    loss = loss_fn(y_train_pred, y_train)
    if t % 10 == 0 and t !=0:
        print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()

In [ ]:
plt.plot(y_train_pred.detach().numpy(), label="Preds")
plt.plot(y_train.detach().numpy(), label="Data")
plt.legend()
plt.show()

plt.plot(hist, label="Training loss")
plt.legend()
plt.show()

In [ ]:
np.shape(y_train_pred)

In [ ]:
# make predictions
y_test_pred = model(x_test)

# invert predictions
y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
y_train = scaler.inverse_transform(y_train.detach().numpy())
y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
y_test = scaler.inverse_transform(y_test.detach().numpy())

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train[:,0], y_train_pred[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(df_ibm)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(y_train_pred)+look_back, :] = y_train_pred

# shift test predictions for plotting
testPredictPlot = np.empty_like(df_ibm)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(y_train_pred)+look_back-1:len(df_ibm)-1, :] = y_test_pred

# plot baseline and predictions
plt.figure(figsize=(15,8))
plt.plot(scaler.inverse_transform(df_ibm))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()